In this notebook the autocorrelations function and time scale is computed for neurons and neurites activity as well as the behaviors
The data is first smoothed or interpolated with different methods.
Autocorrelation fuction can be derived for different versions of the data (smoothed, gap-filled,interpolated, ...) but in the end most plots are generated for not-smoothed raw data. You can generate them again by replacing raw data (MegaData0) with smoothed (MegaData) or intepolated (MegaData_interp) data.

In [ ]:
import h5py
import numpy as np
import Functions.Binning as Bin
import Functions.Regressors as Rss
import copy
import importlib
import matplotlib.pyplot as plt
import Functions.AllFunctions as AllF
import Functions.Make_pandaBinsDataset as MPB
import pandas as pd
from scipy.signal import correlate
from scipy.optimize import curve_fit
import math
import Functions.Autocorr as Aut
from scipy.signal import find_peaks
from scipy.signal import savgol_filter
from numpy.polynomial.polynomial import Polynomial

# Read and Prepare data

In [ ]:
h5 = h5  = h5py.File('MegaData7_7worms_V5.h5', 'r')#'MegaData7_7worms_V2_update.h5'#'MegaData7_copy4.h5'

In [ ]:
outDir = '/mnt/DATA/Mahsa/movies/LongRecordings/Plotting/Plot/AutoCorr/'
res = 300# resolution of the images you save

In [ ]:
L= h5['data_sets_info/lowercam_length']
n_title = [s.decode('utf-8') for s in list(h5['data_sets_info/neuron_names'])]
MegaData0 = np.copy(h5['data/dataraw'])
MegaData0[np.isnan(MegaData0)]=0

In [ ]:
MegaData =copy.deepcopy(MegaData0)

In [ ]:
smooth=True
gapfilled = True
numW=7

In [ ]:
for i in range(55):
    if i >12 and not (i in [27,28,39,40]):
        MegaData[5,:243,i] = 0
        MegaData0[5,:243,i] = 0


for i in range(55):
    MegaData[6,L[6]:,i] = 0
    MegaData0[6,L[6]:,i] = 0

In [ ]:
sm = [2,2,2,3,4,3,3]
g=[5,5,5,7,10,7,7]
if smooth:
    for d in range(7):
        for i in range(55):
            if not (i in [10,11,12]):
                MegaData[d,:L[d],i] = Bin.smooth_vec_long(MegaData0[d,:L[d],i],sm[d],g[d])

MegaData_filled =copy.deepcopy(MegaData0)
importlib.reload(AllF)
if gapfilled:
    for d in range(7):
        for i in range(55):
            if not (i in [10,11,12]):
                MegaData_filled[d,:L[d],i] = AllF.fill_in_gaps(MegaData0[d,:L[d],i],g[d])

MegaData_fs =copy.deepcopy(MegaData0)
if smooth and gapfilled:
    for d in range(7):
        for i in range(55):
            if not (i in [10,11,12]):
                MegaData_fs[d,:L[d],i] = Bin.smooth_vec_long(MegaData_filled[d,:L[d],i],sm[d],g[d])

In [ ]:
MegaData_filled[MegaData_filled==0] = np.nan

In [ ]:
MegaData_interp = copy.deepcopy(MegaData_filled)
for d in range(numW):
    for n in range(55):
        if not (n in [10,11,12]):
            time = MegaData0[d,:L[d],12]
            values = MegaData_filled[d,:L[d],n]
            df = pd.DataFrame({'time': time, 'values': values})
            window_size = 3
            df['values2'] = df['values'].interpolate(method='linear')
            smoothed_values1 = savgol_filter(df['values2'], window_length=5, polyorder=1)
            df['smoothed_values1'] = smoothed_values1
            MegaData_interp[d,:L[d],n] = smoothed_values1

In [ ]:
MegaData_interp[np.isnan(MegaData_interp)]=0
MegaData_filled[np.isnan(MegaData_filled)]=0

## Plot different autocorrelation functions with or without considering 0 values to tes
This cell is not necessary for the PhD plots

In [ ]:
%matplotlib widget

In [ ]:
# Compute autocorrelation
importlib.reload(Aut)
d0=4
n=1
x= copy.deepcopy(MegaData_filled[d0,:L[d0],n])
autocorr = np.correlate(x, x, mode='full')
norm_cross_corr = Aut.norm_cross_corr(x, x, mod='full')#remove mean and divide by std
norm_cross_corr_zer = Aut.norm_cross_corr(x, x, mod='full',zeros=1)#remove mean and divide by std
autocorr_s,normfac =Aut.autocorrelation_s(x)#remove mean and divide by std ony for nonzero values
x[x==0]= np.nan
autocorrnan_corr,lagsnanco2 = Aut.nan_correlate2(x, x, mode='full')# take care of invalid values and the main function for thesis plots
# Center the result
lags = np.zeros(2*len(x)-1)
lags[:len(x)] = -MegaData0[d0,:L[d0],12][::-1]+MegaData0[d0,0,12]
lags[len(x):] = MegaData0[d0,1:L[d0],12]- MegaData0[d0,0,12]


In [ ]:
AllF.plot_autocorr(lags, autocorr/np.max(autocorr),titre='numpy correlate')
AllF.plot_autocorr(lags, norm_cross_corr,titre='normalized signal')
AllF.plot_autocorr(lags, norm_cross_corr_zer,titre='normalized signal_nonzero')
AllF.plot_autocorr(lags, autocorrnan_corr  ,titre='nancorr_2')
AllF.plot_autocorr(lags, autocorr_s/autocorr_s[len(x)-1]  ,titre='nancorr_s')

In [ ]:
#plot multiple correlations on top of each other
plt.figure(figsize=(10,4))
plt.stem(lags,autocorr_s/autocorr_s[len(x)-1],'red')
plt.stem(lags, autocorrnan_corr)#, use_line_collection=True)
hl=1/np.e
plt.axhline(y= hl, color='green', linestyle='--')
plt.title("Autocorrelation of the Signal")
plt.xlabel("Lag")
plt.ylabel("Autocorrelation")
print(len(np.nonzero(x)[0])/L[d0])
plt.grid()
plt.show()

In [ ]:
#fit an functions to autocorr
CorrFunc = copy.deepcopy(autocorrnan_corr)#which autocorrelation function to fit
thresh=100
# Generate example autocorrelation data
time = lags[len(x)-1:]
#valid= np.nonzero(~np.isnan(CorrFunc[len(x)-1:]))[0]
#if you want to fit oly to the beginning
valid0= np.nonzero(~np.isnan(CorrFunc[len(x)-1:]))[0]
valid = [i for i in valid0 if time[i] < thresh]
valid = np.array(valid)
time=time[:int(np.max(valid))+1]
# Fit different functions to the autocorrelation
popt, pcov = curve_fit(Aut.exponential_decay, time[valid], CorrFunc[len(x)-1+valid], p0=[1, 20])  # Initial guess for A and tau_0
fitted_A, fitted_tau_0 = popt
popt2, pcov = curve_fit(Aut.exponential1_decay, time[valid], CorrFunc[len(x)-1+valid], p0=[20])  # Initial guess for tau_0
popt_gauss, pcov5 = curve_fit(Aut.gaussian_decay, time[valid], CorrFunc[len(x)-1+valid], p0=[1, 20])
gauss1, gauss0 = popt_gauss
popt_stretch, pcov3 = curve_fit(Aut.stretched_exponential_decay, time[valid],
                                    CorrFunc[len(x)-1+valid], p0=[1, 20, 1],maxfev=5000)
se2,se0,se1= popt_stretch

popt_power, pcov4 = curve_fit(Aut.power_law_decay, time[valid], CorrFunc[len(x)-1+valid], p0=[1, 1, 1],bounds=([0, 0, 0],                                                                                     [np.inf, np.inf, np.inf]),maxfev=5000)
power_law2, power_law0, power_law1= popt_power

popt_double, pcov2 = curve_fit(Aut.double_exponential_decay, time[valid], CorrFunc[len(x)-1+valid], p0=[1, 20, 1, 1],maxfev=5000)
de2,de0,de3,de10= popt_double


print(f"Fitted Amplitude (A): {fitted_A:.3f}")
print(f"Fitted Time Constant (tau_0): {fitted_tau_0:.3f}")



In [ ]:
# Plot the results
fs=15
lwidth = 4
al=0.8
fig = plt.figure(figsize=(12,6))
plt.scatter(time[valid], CorrFunc[len(x)-1+valid], label="Autocorrelation", color="blue",s=100,marker='*')
plt.plot(time, Aut.exponential_decay(time, *popt), label=f"Exp,τ:{fitted_tau_0:.2f}", color="red",lw=lwidth,alpha = al)
plt.plot(time, Aut.exponential1_decay(time, *popt2), label=f"Exp1,τ:{float(popt2):.2f}", color="green",lw=lwidth,alpha = al)
plt.plot(time, Aut.gaussian_decay(time, *popt_gauss), label=f"Gaussian,τ:{gauss0:.2f}", color="orange",lw=lwidth,alpha = al)
plt.plot(time, Aut.stretched_exponential_decay(time, *popt_stretch), label=f"stretched,Exp-τ:{se0:.2f}", color="cyan",lw=lwidth,alpha = al)
plt.plot(time, Aut.power_law_decay(time, *popt_power), label=f"power-law,τ:{power_law0:.2f}", color="magenta",lw=lwidth,alpha = al)
plt.plot(time, np.array(Aut.double_exponential_decay(time, *popt_double)), label=f"double Exp,τ1:{de0:.2f}-τ2:{de10:.2f}",
         color="purple",lw=lwidth,alpha = al)
plt.xlabel("Lags[sec]",fontsize=fs)
plt.ylabel("Autocorrelation")
plt.xscale('log')   # log x-axis
#plt.yscale('log')   # log y-axis
plt.xticks(fontsize=fs,rotation=0)
hl=1/np.e
plt.axhline(y= hl, color='green', linestyle='--')
plt.yticks(fontsize=fs)
plt.title("Different fits")
plt.legend()
plt.grid()
fig.savefig(f"{outDir}SingleAutoCorr0_W{d0}_n{n}_exponly100alllog.pdf", dpi=res, bbox_inches='tight')
fig.savefig(f"{outDir}SingleAutoCorr_W{d0}_n{n}_exponly100allllog.png", dpi=res, bbox_inches='tight')
plt.show()

## plot different stretches

In [ ]:
importlib.reload(Aut)
RR = 100
d0=5
n=2
x= copy.deepcopy(MegaData0[d0,:L[d0],n])
start_indices,end_indices = AllF.get_nonzero_start_end_long(x,thresh=3)
stretches =[(start_indices[i],end_indices[i]) for i in  range(len(start_indices))]
final_matrix,total_weight = Aut.averaging_crosscorr_diff_stretches(x,x,stretches,threshold=RR)
lags0 = np.arange(-RR + 1, RR)
lags0[:RR] = -MegaData0[d0,:RR,12][::-1]
lags0[RR:] = MegaData0[d0,1:RR,12]

In [ ]:
for g in range(final_matrix.shape[0]):
    AllF.plot_autocorr(lags0, final_matrix[g,:-1])

In [ ]:
d=5
n=4
x= copy.deepcopy(MegaData0[d,:L[d],n])
x[x==0]= np.nan
autocorrnan2,lagsnan2 = Aut.nan_correlate2(x,x)#correlate(x, x, mode='full', method='auto')
lags = np.arange(-len(x) + 1, len(x))

lags[:len(x)] = -MegaData0[d0,:L[d0],12][::-1]
lags[len(x):] = MegaData0[d0,1:L[d0],12]
AllF.plot_autocorr(lagsnan2, autocorrnan2)
AllF.plot_autocorr(lags, autocorrnan2)

# Neurons Autocorrelations

In [ ]:
numW=7
num_neu = 7
FullAutocorr = np.zeros((numW,2*np.max(L)-1,num_neu))
FullLags = np.zeros((numW,2*np.max(L)-1,num_neu))
Time_constant = np.zeros((numW,num_neu,2))#0time,1scale
double_exponential = np.zeros((numW,num_neu,4))#0time,1scale
stretched_exponential = np.zeros((numW,num_neu,3))#
power_law = np.zeros((numW,num_neu,3))#
gaussian = np.zeros((numW,num_neu,2))#
threshTimes = np.zeros((numW,num_neu))#

## Try autocorrelation on different versions of the data (smoothed, gap-filled, interpolated, ...)
### the last one, raw data, is the one used for plots mostly

In [ ]:
importlib.reload(Aut)
lim=100
# compute autocorr and time constants for interpolated,raw, and gap-filled dataset
FullLags,FullAutocorr,Time_constant,exp1,gaussian,threshTimes = Aut.get_time_scale(MegaData_interp[:numW,:,:7],L,MegaData0[:numW,:,12])
FullLags_0,FullAutocorr_0,Time_constant_0,exp1_0,gaussian_0,threshTimes_0 = Aut.get_time_scale(MegaData0[:numW,:,:7],L,MegaData0[:numW,:,12])
FullLags_f,FullAutocorr_f,Time_constant_f,exp1_f,gaussian_f,threshTimes_f = Aut.get_time_scale(MegaData_filled[:numW,:,:7],L,MegaData0[:numW,:,12])
#compute time constants for different fitting curves on raw data's autocorrelation, compute fits for first lim seconds
FullLags_a,FullAutocorr_a,Time_constant_a,exp1_a,gaussian_a,stretched_exponential_a,power_law_a,threshTimes_a = Aut.get_time_scale(MegaData0[:numW,:,:7],
                                            L,MegaData0[:numW,:,12],allfit=True,thresh=lim)

In [ ]:
d0=5# number of worm
n0=4#number of neuron
fig = Aut.plot_autocorr(FullLags[d0,:2*L[d0],n0], FullAutocorr[d0,:2*L[d0],n0])
fig = Aut.plot_autocorr(FullLags_a[d0,:2*L[d0],n0], FullAutocorr_a[d0,:2*L[d0],n0])

## plot AutoCorr with exponential fit

In [ ]:
WormLabel = ['W0','W1','W2','W3','W4','W5','W6']
Neur_lab = ["RIAL","RIAR","RIML","RIMR","RIBL","RIBR",'Sens.']

In [ ]:
n,d = 4,3
plt.figure(figsize=(10,4))
temp = FullLags[d,L[d]:2*L[d]-1,n]
autocorr = FullAutocorr[d,L[d]:2*L[d]-1,n]
popt2 = Time_constant_a[d,n,1], Time_constant_a[d,n,0] 
plt.scatter(temp, autocorr, label="Autocorrelation (data)", color="blue")
plt.plot(temp, Aut.exponential_decay(temp, *popt2), label="Fitted Exponential", color="red")
plt.xlabel("Lag")
plt.ylabel("Autocorrelation")
plt.title("Exponential Fit to Autocorrelation")
hl=1/np.e
plt.axhline(y= hl, color='green', linestyle='--')#, label='y = 1.96')
plt.legend()
plt.grid()
plt.show()

In [ ]:
n,d = 4,5
fig = plt.figure(figsize=(10,4))
temp = FullLags[d,L[d]:2*L[d]-1,n]
autocorr = FullAutocorr[d,L[d]:2*L[d]-1,n]
popt = exp1_a[d,n], 
plt.scatter(temp, autocorr, label="Autocorrelation (data)", color="blue")
plt.plot(temp, Aut.exponential1_decay(temp, *popt), label="Fitted Exponential", color="red")
plt.xlabel("Lag")
plt.ylabel("Autocorrelation")
plt.title("Exponential Fit to Autocorrelation")
hl=1/np.e
plt.axhline(y= hl, color='green', linestyle='--')#, label='y = 1.96')
plt.legend()
plt.grid()
plt.show()

In [ ]:
exp1_a[2,2:6]= np.nan

In [ ]:
lim =100
fig = Aut.plot_parameters_bar(threshTimes_a,Neur_lab, paramName=f'τ - thresh-{lim}mean',fs = 20)
fig.savefig(f"{outDir}ave_tau_thresh_bar_mean{lim}.pdf", dpi=res, bbox_inches='tight')
fig.savefig(f"{outDir}ave_tau_thresh_bar_mean{lim}.png", dpi=res, bbox_inches='tight')
fig = Aut.plot_parameters_bar(threshTimes_a,Neur_lab, paramName=f'τ - thresh-{lim}med',fs = 20,median=1)
fig.savefig(f"{outDir}ave_tau_thresh_bar_med{lim}.pdf", dpi=res, bbox_inches='tight')
fig.savefig(f"{outDir}ave_tau_thresh_bar_med{lim}.png", dpi=res, bbox_inches='tight')

In [ ]:
fig = Aut.plot_parameters_bar(exp1_a,Neur_lab, paramName=f'τ - exp1-{lim}mean',fs = 20)
fig.savefig(f"{outDir}ave_tau_e1_bar_mean{lim}.pdf", dpi=res, bbox_inches='tight')
fig.savefig(f"{outDir}ave_tau_e1_bar_mean{lim}.png", dpi=res, bbox_inches='tight')
fig = Aut.plot_parameters_bar(exp1_a,Neur_lab, paramName=f'τ - exp1-{lim}med',fs = 20,median=1)
fig.savefig(f"{outDir}ave_tau_e1_bar_med{lim}.pdf", dpi=res, bbox_inches='tight')
fig.savefig(f"{outDir}ave_tau_e1_bar_med{lim}.png", dpi=res, bbox_inches='tight')

In [ ]:
FullLags_a.shape

In [ ]:
importlib.reload(Aut)

fig = Aut.plot_AllneuronsFit(FullLags_a,FullAutocorr_a,Neur_lab,Time_constant,L,exp=0,limit=lim)
fig.savefig(f"{outDir}all_neuronsFit_exp{lim}.pdf", dpi=res, bbox_inches='tight')
fig.savefig(f"{outDir}all_neuronsFit_exp{lim}.png", dpi=res, bbox_inches='tight')
fig = Aut.plot_AllneuronsFit(FullLags_a,FullAutocorr_a,Neur_lab,exp1_a,L,exp=1,limit=lim)
fig.savefig(f"{outDir}all_neuronsFit_exp1{lim}.pdf", dpi=res, bbox_inches='tight')
fig.savefig(f"{outDir}all_neuronsFit_exp1{lim}.png", dpi=res, bbox_inches='tight')
fig = Aut.plot_AllneuronsFit_thresh(FullLags_a,FullAutocorr_a,Neur_lab,exp1_a,threshTimes_a,L,exp=1,limit=lim)
fig.savefig(f"{outDir}all_neuronsFit_thresh{lim}.pdf", dpi=res, bbox_inches='tight')
fig.savefig(f"{outDir}all_neuronsFit_thresh{lim}.png", dpi=res, bbox_inches='tight')

In [ ]:
# Adjust figure size and create 7x7 subplots
fig, axes = plt.subplots(7, 7, figsize=(20, 10), sharex=True, sharey=True)
fig.subplots_adjust(hspace=0.4, wspace=0.4)
Neur_lab = ["RIAL","RIAR","RIML","RIMR","RIBL","RIBR",'Sensory']
# Loop over all combinations of n and d
for n in range(7):
    for d in range(7):
        ax = axes[d, n]  # Access the subplot
        
        # Extract data for current n and d
        temp = FullLags[d, L[d]:2*L[d]-1, n]
        autocorr = FullAutocorr[d, L[d]:2*L[d]-1, n]
        popt = exp1[d,n],
        
        # Plot scatter and fitted curve
        ax.scatter(temp, autocorr, label="Data", color="blue", alpha=0.6)
        ax.plot(temp, Aut.exponential1_decay(temp, *popt), label="Fit", color="red", alpha=0.8,lw=4)
        hl=1/np.e
        ax.axhline(y= hl, color='green', linestyle='--')
        # Add titles and grid
        ax.set_title(Neur_lab[n]+f", W={d}", fontsize=15)
        ax.grid(alpha=0.5)
        ax.set_ylim(-0.1, 1)
        ax.set_xlim(0,right= 300)
        tau_value = exp1[d, n]
        ax.text(0.3, 0.7, f"τ={tau_value:.2f}", transform=ax.transAxes, fontsize=10, color="black")
        # Remove ticks for better readability
        ax.tick_params(axis='both', which='major', labelsize=15)

# Set shared labels
fig.text(0.5, 0.04, 'Lag', ha='center', fontsize=20)
fig.text(0.04, 0.5, 'Autocorrelation', va='center', rotation='vertical', fontsize=20)
fig.savefig(f"{outDir}all_neuronsFit_interpolated_exp1.pdf", dpi=res, bbox_inches='tight')
fig.savefig(f"{outDir}all_neuronsFit_interpolated_exp1.png", dpi=res, bbox_inches='tight')
# Show the p

# Test time scales of neurites Only

In [ ]:
n_tit = ["sensory","nrDL","loopL","nrDR","loopR","nrVL","nrVR"]
neu = [6,7,8,27,28,39,40]#[41,42,43,45,46,47,48]
beh_lab = [str(i) for i in neu]

In [ ]:
importlib.reload(Aut)
FullLags_ax0,FullAutocorr_ax0,Time_constant_ax0,exp1_ax0,gaussian_ax0,threshTimes_ax0  = Aut.get_time_scale(MegaData0[:numW,:,neu],
                                                                                            L,MegaData0[:numW,:,12],neurite=True)
FullLags_ax,FullAutocorr_ax,Time_constant_ax,exp1_ax,gaussian_ax,threshTimes_ax  = Aut.get_time_scale(MegaData_interp[:numW,:,neu],
                                                                                        L,MegaData0[:numW,:,12],neurite=True)

In [ ]:
importlib.reload(Aut)
fig1 = Aut.plot_parameters_heat(exp1_ax0,n_tit, WormLabel, paramName='τ Value',fs = 15,titre = 'τ - exp1')
fig2 = Aut.plot_parameters_heat(exp1_ax,n_tit, WormLabel, paramName='τ Value',fs = 15,titre = 'τ - exp1-intp')
fig3 = Aut.plot_parameters_heat(Time_constant_ax0[:,:,0],n_tit, WormLabel, paramName='τ- gf ',fs = 15,titre = 'τ - exp ')


In [ ]:
worms_to_select = [0,1,2,3,5] 
WormLabel_sub = [WormLabel[i] for i in worms_to_select]
fig = Aut.plot_parameters_heat(exp1_ax0[worms_to_select,:],n_tit, WormLabel_sub, paramName='τ Value',fs = 15,titre = 'τ - exp1-ns')
fig = Aut.plot_parameters_heat(exp1_ax[worms_to_select,:],n_tit, WormLabel_sub, paramName='τ Value',fs = 15,titre = 'τ - exp1-intp')
fig = Aut.plot_parameters_heat(Time_constant_ax0[worms_to_select,:,0],n_tit, WormLabel_sub, paramName='τ ',fs = 15,titre = 'τ - exp - ns')


In [ ]:
fig = Aut.plot_AllneuronsFit(FullLags_ax0,FullAutocorr_ax0,n_tit,exp1_ax0,L,exp=1)
fig.savefig(f"{outDir}all_neuritesFit_exp1.pdf", dpi=res, bbox_inches='tight')
fig.savefig(f"{outDir}all_neuritesFit_exp1.png", dpi=res, bbox_inches='tight')
fig = Aut.plot_AllneuronsFit_thresh(FullLags_ax0,FullAutocorr_ax0,n_tit,exp1_ax0,threshTimes_ax0,L,exp=1)
fig.savefig(f"{outDir}all_neuritesFit_exp1_thresh.pdf", dpi=res, bbox_inches='tight')
fig.savefig(f"{outDir}all_neuritesFit_exp1_thresh.png", dpi=res, bbox_inches='tight')

# Test for behavior

In [ ]:
neu = [15,16,38,46,26]
beh_lab = [str(i) for i in neu]

In [ ]:
importlib.reload(Aut)
lim=100
FullLags_b0,FullAutocorr_b0,Time_constant_b0,exp1_b0,gaussian_bf,threshTimes_b0 = Aut.get_time_scale(MegaData0[1:7,:,neu],L[1:],           
                                                                                MegaData0[1:7,:,12])
FullLags_bf,FullAutocorr_bf,Time_constant_bf,exp1_bf,gaussian_bf,threshTimes_bf = Aut.get_time_scale(MegaData_filled[1:7,:,neu],L[1:],
                                                                                MegaData0[1:7,:,12])
FullLagsb_inter,FullAutocorrb_inter,Time_constantb_inter,exp1b_inter,gaussianb_inter,stretched_exponentialb_inter,power_lawb_inter,threshTimesb_inter = Aut.get_time_scale(MegaData_interp[1:7,:,neu],
                                                                                    L[1:],MegaData0[1:7,:,12],allfit=True,thresh=lim)
FullLagsb_lim,FullAutocorrb_lim,Time_constantb_lim,exp1b_lim,gaussianb_lim,stretched_exponentialb_lim,power_lawb_lim,threshTimesb_lim = Aut.get_time_scale(MegaData0[1:7,:,neu],
                                                                                                    L[1:],MegaData0[1:7,:,12],allfit=True,thresh=lim)

In [ ]:
importlib.reload(Aut)
WormLabel = ['W0','W1','W2','W3','W4','W5','W6']
fig = Aut.plot_parameters_heat(exp1_b0,beh_lab, WormLabel[1:], paramName='τ Value',fs = 20,titre = 'τ - exp1')
fig = Aut.plot_parameters_heat(Time_constant_b0[:,:,0],beh_lab, WormLabel[1:], paramName='τ Value',fs = 20,titre = 'τ - exp')



In [ ]:
fig = Aut.plot_parameters_bar(exp1_b0,['speed','velocity','head angle','center angle','head curvature'], 
                              paramName='τ Value-exp1',fs = 20,r=30)
fig.savefig(f"{outDir}tau_behaviors_exp1.pdf", dpi=res, bbox_inches='tight')
fig.savefig(f"{outDir}tau_behaviors_exp1.png", dpi=res, bbox_inches='tight')
fig = Aut.plot_parameters_bar(threshTimes_b0,['speed','velocity','head angle','center angle','head curvature'], 
                              paramName='τ Value-thresh',fs = 20,r=30)
fig.savefig(f"{outDir}tau_behaviors_thresh.pdf", dpi=res, bbox_inches='tight')
fig.savefig(f"{outDir}tau_behaviors_thresh.png", dpi=res, bbox_inches='tight')

In [ ]:
importlib.reload(Aut)
WormLabel = ['W0','W1','W2','W3','W4','W5','W6']

fig =Aut.plot_parameters_heat(exp1_bf,beh_lab, WormLabel[1:], paramName='τ Value',fs = 20,titre = 'τ - exp1, gapfilled')
fig = Aut.plot_parameters_heat(Time_constant_bf[:,:,0],beh_lab, WormLabel[1:], paramName='τ ',fs = 20,titre = 'τ - exp, gapfilled')



In [ ]:
# Adjust figure size and create 7x7 subplots
fig, axes = plt.subplots(6, 5, figsize=(20, 10), sharex=True, sharey=True)
fig.subplots_adjust(hspace=0.4, wspace=0.4)
Neur_lab = ["RIAL","RIAR","RIML","RIMR","RIBL","RIBR",'Sensory']
# Loop over all combinations of n and d
for n in range(5):
    for d in range(6):
        ax = axes[d, n]  # Access the subplot
        d_real = d+1
        # Extract data for current n and d
        temp = FullLags_b0[d, L[d_real]-1:2*L[d_real]-1, n]
        autocorr = FullAutocorr_b0[d, L[d_real]-1:2*L[d_real]-1, n]
        popt = Time_constant_b0[d, n, 1], Time_constant_b0[d, n, 0]#exp1_ax0[d,n],
        popt1 = exp1_b0[d,n],
        # Plot scatter and fitted curve
        ax.scatter(temp, autocorr, label="Data", color="blue", alpha=0.6)
        ax.plot(temp, Aut.exponential_decay(temp, *popt), label="Fit", color="red", alpha=0.8,lw=4)
        ax.plot(temp, Aut.exponential1_decay(temp, *popt1), label="Fit", color="green", alpha=0.8,lw=4)
        hl=1/np.e
        ax.axhline(y= hl, color='green', linestyle='--')
        # Add titles and grid
        ne= neu[n]
        ax.set_title(f"b={ne}, W={d_real}", fontsize=15)
        ax.grid(alpha=0.5)
        ax.set_ylim(-0.1, 1)
        #ax.set_xlim(-2,50)
        ax.set_xscale('log') 
        tau_value = Time_constant_b0[d, n, 0]#exp1_ax0[d, n]
        ax.text(0.3, 0.7, f"τ={tau_value:.2f}", transform=ax.transAxes, fontsize=10, color="black")
        # Remove ticks for better readability
        ax.tick_params(axis='both', which='major', labelsize=15)

# Set shared labels
fig.text(0.5, 0.04, 'Lag', ha='center', fontsize=20)
fig.text(0.04, 0.5, 'Autocorrelation', va='center', rotation='vertical', fontsize=20)
fig.savefig(f"{outDir}All_behaviorsFit_exp_logs.pdf", dpi=res, bbox_inches='tight')
fig.savefig(f"{outDir}All_behaviorsFit_exp_logs.png", dpi=res, bbox_inches='tight')
plt.show()

# Time scale of all neurons and neurites

In [ ]:
numW=7
num_neu = 13
FullAutocorr = np.zeros((numW,2*np.max(L)-1,num_neu))
FullLags = np.zeros((numW,2*np.max(L)-1,num_neu))
Time_constant = np.zeros((numW,num_neu,2))#0time,1scale
double_exponential = np.zeros((numW,num_neu,4))#0time,1scale
stretched_exponential = np.zeros((numW,num_neu,3))#
power_law = np.zeros((numW,num_neu,3))#
gaussian = np.zeros((numW,num_neu,2))#

In [ ]:
n_tit = ["RIAL","RIAR","RIML","RIMR","RIBL","RIBR","Sens.","nrDL","loopL","nrDR","loopR","nrVL","nrVR"]
neu = [0,1,2,3,4,5,6,7,8,27,28,39,40]#[41,42,43,45,46,47,48]
beh_lab = [str(i) for i in neu]

In [ ]:
importlib.reload(Aut)
lim=300
FullLags_All0,FullAutocorr_All0,Time_constant_All0,exp1_All0,gaussian_All0,threshTimes_All0 = Aut.get_time_scale(MegaData0[:numW,:,neu],L,
                                                                                                MegaData0[:numW,:,12])
FullLags_All,FullAutocorr_All,Time_constant_All,exp1_All,gaussian_All,threshTimes_All = Aut.get_time_scale(MegaData_interp[:numW,:,neu],L,
                                                                                           MegaData0[:numW,:,12])
FullLags_Alllim,FullAutocorr_Alllim,Time_constant_Alllim,exp1_Alllim,gaussian_Alllim,stretched_exponential_Alllim,power_law_Alllim, threshTimes_Alllim = Aut.get_time_scale(MegaData0[:numW,:,neu],L,
                                                                                MegaData0[:numW,:,12],allfit=True,thresh=lim)

In [ ]:
worms_to_select = [0,1,3,4,5,6] 
WormLabel_sub = [WormLabel[i] for i in worms_to_select]


In [ ]:
importlib.reload(Aut)
fig = Aut.plot_parameters_bar(exp1_All0[worms_to_select,:],n_tit, paramName=f'τ exp1-ns{lim}',fs = 15,r=45)
fig.savefig(f"{outDir}tau_AllNeurons_exp1{lim}.pdf", dpi=res, bbox_inches='tight')
fig.savefig(f"{outDir}tau_AllNeurons_exp1{lim}.png", dpi=res, bbox_inches='tight')
fig = Aut.plot_parameters_bar(threshTimes_All0[worms_to_select,:],n_tit, paramName=f'τ thresh-ns{lim}',fs = 15,r=45)
fig.savefig(f"{outDir}tau_AllNeurons_thresh{lim}.pdf", dpi=res, bbox_inches='tight')
fig.savefig(f"{outDir}tau_AllNeurons_thresh{lim}.png", dpi=res, bbox_inches='tight')
fig = Aut.plot_parameters_bar(Time_constant_All0[worms_to_select,:,0],n_tit, paramName=f'τ exp-ns{lim}',fs = 15,r=45)
fig.savefig(f"{outDir}tau_AllNeurons_exp{lim}.pdf", dpi=res, bbox_inches='tight')
fig.savefig(f"{outDir}tau_AllNeurons_exp{lim}.png", dpi=res, bbox_inches='tight')

In [ ]:
fig = Aut.plot_AllneuronsFit_thresh(FullLags_All0,FullAutocorr_All0,n_tit,exp1_All0,threshTimes_All0,L,exp=1,limit=lim)
fig.savefig(f"{outDir}all_AllNeuronsFit_thresh{lim}.pdf", dpi=res, bbox_inches='tight')
fig.savefig(f"{outDir}all_AllNeuronsFit_thresh{lim}.png", dpi=res, bbox_inches='tight')